<a href="https://colab.research.google.com/github/justinfmccarty/bipv_database/blob/main/extract_bipv_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Eiffert and Kiss - Building Integrated Photovoltaic Design Book

Eiffert, Patrina, and Gregory J Kiss. “Building-Integrated Photovoltaic Designs for Commercial and Institutional Structures A Sourcebook for Architects,” 2000, 92.


In [ ]:
!pip install pdfplumber

In [207]:
import pdfplumber
import pandas as pd
from google.colab import files

In [40]:
# url = r'https://github.com/justinfmccarty/bipv_database/raw/main/Eiffert%20and%20Kiss%20-%20Building-Integrated%20Photovoltaic%20Designs%20for%20Comme.pdf'
url = r'https://github.com/justinfmccarty/bipv_database/raw/main/Pages%20from%20Eiffert%20and%20Kiss%20-%20Building-Integrated%20Photovoltaic%20Design.pdf'
import os
import urllib.request
urllib.request.urlretrieve(url, 'eiffert_and_kiss.pdf')

('eiffert_and_kiss.pdf', <http.client.HTTPMessage at 0x7fbc3ec1d410>)

In [ ]:
with pdfplumber.open('/content/eiffert_and_kiss.pdf') as pdf:
    total_pages = range(len(pdf.pages))

In [ ]:
for i in total_pages:
      print(i)

In [182]:
txt = ['Approx.', '3,800']

for s in txt:
  if s[0].isdigit():
    print(s)

3,800


In [208]:
df_all = []

for i in total_pages:
  with pdfplumber.open('/content/eiffert_and_kiss.pdf') as pdf:
      first_page = pdf.pages[i]
      text = first_page.extract_text(x_tolerance=0, y_tolerance=3)
      text_file = open(f"Output{i}.txt", "w")
      text_file.write(text)
      text_file.close()

  data_dict = dict()

  a_file = open(f"Output{i}.txt")
  lines = a_file.readlines()
  lines
  for x, line in enumerate(lines):
    # get name
    if x==0:
      data_dict['name'] = line[2:]

    # get year completed
    if "Date Completed:" in line:
      data_dict['year'] = line.split(' ')[-1]
    
    # get module type
    if "PV Product:" in line:
      data_dict['module'] = line.split(':')[1]

    # get cell type
    if "Cell" in line:
      data_dict['cell'] = line.split(':')[1]

    # get connect type
      # if "ction:" in line:
      #   data_dict['connection'] = line.split(':')[1]

    # get output kWh annual
    if "Output:" in line:
      if 'h' in line:
        output_str = line.split(':')[1].split(' kWh')[0]
        if ',' in output_str:
          output = float(output_str.replace(',',''))
        else:
          output = float(output_str)
        data_dict['output'] = output
      else:
        data_dict['output'] = 'na'

    # get and convert coverage
    if "Area:" in line:
      area_str = line.split(':')[1].split(' ft')[0]      
      if 'm' in area_str:
        area = area_str.split(' m')[0]
        if ',' in area:
          area = float(area.replace(',',''))
        else:
          area = float(area)
      elif ',' in area_str:
        if 'pprox' in area_str:
          area_str = area_str.split('. ')[1]
          area = float(area_str.replace(',',''))*0.092903
        else:
          area = float(area_str.replace(',',''))*0.092903
      
      else:
        area = float(area_str)*0.092903
      data_dict['coverage'] = area

    # get rating
    if "kWp" in line:
      if 'nve' in line:
        data_dict['rating'] = 'na'
      else:
        rating_str = line.split(':')[1]
        rating = rating_str.replace('kWp','')
        data_dict['rating'] = rating

  page_data = pd.DataFrame.from_dict(data_dict, orient='index').transpose()
  df_all.append(page_data)
  

pd.concat(df_all).to_csv('eiffert_and_kiss.csv')
files.download('eiffert_and_kiss.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>